In [1]:
import pandas as pd

In [2]:
matches = pd.read_csv("matches_by_teams.csv")

In [3]:
#variable types
matches.dtypes

match_id                               object
color                                  object
team_id                                object
team_slug                              object
team_name                              object
team_region                            object
core_shots                            float64
core_goals                            float64
core_saves                            float64
core_assists                          float64
core_score                            float64
core_shooting_percentage              float64
boost_bpm                             float64
boost_bcpm                            float64
boost_avg_amount                      float64
boost_amount_collected                float64
boost_amount_stolen                   float64
boost_amount_collected_big            float64
boost_amount_stolen_big               float64
boost_amount_collected_small          float64
boost_amount_stolen_small             float64
boost_count_collected_big         

In [4]:
matches["color_code"] = matches["color"].astype("category").cat.codes

In [5]:
matches["region_code"] = matches["team_region"].astype("category").cat.codes

In [6]:
matches["target"] = (matches["winner"] == True).astype("int")

In [7]:
matches

,match_id,color,team_id,team_slug,team_name,team_region,core_shots,core_goals,core_saves,core_assists,...,positioning_time_offensive_half,positioning_time_behind_ball,positioning_time_in_front_ball,demo_inflicted,demo_taken,score,winner,color_code,region_code,target
0,6159ad3d143c37878b2384a9,blue,6020bc8ef1e4807cc700391a,https://octane.gg/teams/391a-ground-zero-gaming,GROUND ZERO GAMING,Oceania,34.0,9.0,8.0,7.0,...,1237.05,2339.78,865.27,6.0,7.0,3.0,True,0,5,1
1,6159ad3d143c37878b2384a9,orange,614c8930f8090ec745286474,https://octane.gg/teams/6474-ranga-roundup,RANGA ROUNDUP,Oceania,15.0,5.0,21.0,3.0,...,998.99,2318.26,890.30,7.0,6.0,0.0,False,1,5,0
2,6159ad3d143c37878b2384aa,blue,6020bc70f1e4807cc70023fb,https://octane.gg/teams/23fb-renegades,RENEGADES,Oceania,46.0,21.0,10.0,21.0,...,1505.61,2616.06,862.25,11.0,7.0,3.0,True,0,5,1
3,6159ad3d143c37878b2384aa,orange,6156d4db143c37878b238367,https://octane.gg/teams/8367-trident-esports,TRIDENT ESPORTS,Oceania,16.0,4.0,16.0,3.0,...,859.84,2355.02,1105.02,7.0,11.0,0.0,False,1,5,0
4,6159ad3d143c37878b2384ab,blue,6020bc8ef1e4807cc7003915,https://octane.gg/teams/3915-dire-wolves,DIRE WOLVES,Oceania,46.0,12.0,13.0,9.0,...,1718.97,3444.64,1174.96,13.0,8.0,3.0,True,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10589,62e02371da9d7ca1c7bb2f61,orange,6020bd98f1e4807cc700dc74,https://octane.gg/teams/dc74-team-bds,TEAM BDS,Europe,61.0,18.0,29.0,14.0,...,2630.25,5280.76,1550.99,19.0,13.0,4.0,True,1,2,1
10590,62e02371da9d7ca1c7bb2f62,blue,6020bc70f1e4807cc70023a5,https://octane.gg/teams/23a5-g2-esports,G2 ESPORTS,North America,46.0,14.0,38.0,12.0,...,2162.11,4417.79,1616.17,22.0,15.0,4.0,True,0,4,1
10591,62e02371da9d7ca1c7bb2f62,orange,605aca6853a71a78eacbc155,https://octane.gg/teams/c155-faze-clan,FAZE CLAN,North America,50.0,5.0,26.0,4.0,...,2281.36,4300.44,1712.41,15.0,22.0,2.0,False,1,4,0
10592,62e02371da9d7ca1c7bb2f63,blue,6020bd98f1e4807cc700dc74,https://octane.gg/teams/dc74-team-bds,TEAM BDS,Europe,36.0,10.0,19.0,8.0,...,1685.25,3816.52,1227.65,11.0,21.0,4.0,True,0,2,1


In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [10]:
train = matches[matches["match_id"] < '62a3988cda9d7ca1c7bb22ba']

In [11]:
test = matches[matches["match_id"] > '62a3988cda9d7ca1c7bb22ba']

In [12]:
predictors = ["color_code", "region_code", "core_shots", "core_shooting_percentage", "core_saves", "demo_inflicted"]

In [13]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [14]:
preds = rf.predict(test[predictors])

In [15]:
from sklearn.metrics import accuracy_score

In [16]:
acc = accuracy_score(test["target"], preds)

In [17]:
acc

0.8021978021978022

In [18]:
combined = pd.DataFrame(dict(actual=test["target"], prediction = preds))

In [19]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,72,19
1,17,74


In [20]:
from sklearn.metrics import precision_score

In [21]:
precision_score(test["target"], preds)

np.float64(0.7956989247311828)

In [22]:
grouped_matches = matches.groupby("team_name")

In [23]:
group = grouped_matches.get_group("TEAM BDS")

In [24]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("match_id")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [25]:
cols = ["core_shots", "core_saves", "core_goals", "core_score", "core_shooting_percentage", "movement_time_supersonic_speed", "demo_inflicted"]
new_cols = [f"{c}_rolling" for c in cols]

In [26]:
new_cols

['core_shots_rolling',
 'core_saves_rolling',
 'core_goals_rolling',
 'core_score_rolling',
 'core_shooting_percentage_rolling',
 'movement_time_supersonic_speed_rolling',
 'demo_inflicted_rolling']

In [27]:
rolling_averages(group, cols, new_cols)

,match_id,color,team_id,team_slug,team_name,team_region,core_shots,core_goals,core_saves,core_assists,...,color_code,region_code,target,core_shots_rolling,core_saves_rolling,core_goals_rolling,core_score_rolling,core_shooting_percentage_rolling,movement_time_supersonic_speed_rolling,demo_inflicted_rolling
972,61671750143c37878b238eab,blue,6020bd98f1e4807cc700dc74,https://octane.gg/teams/dc74-team-bds,TEAM BDS,Europe,24.0,9.0,15.0,9.0,...,0,2,1,54.000000,26.000000,10.000000,6264.333333,18.749729,795.106667,19.000000
2180,618005c6f8090ec74528a42a,blue,6020bd98f1e4807cc700dc74,https://octane.gg/teams/dc74-team-bds,TEAM BDS,Europe,30.0,8.0,11.0,5.0,...,0,2,1,44.666667,23.000000,10.333333,5505.333333,26.121524,601.606667,18.666667
2200,618005c6f8090ec74528a434,blue,6020bd98f1e4807cc700dc74,https://octane.gg/teams/dc74-team-bds,TEAM BDS,Europe,20.0,4.0,16.0,4.0,...,0,2,1,39.000000,18.000000,9.333333,4927.000000,27.208995,568.766667,17.333333
2213,618005c6f8090ec74528a43a,orange,6020bd98f1e4807cc700dc74,https://octane.gg/teams/dc74-team-bds,TEAM BDS,Europe,27.0,6.0,18.0,4.0,...,1,2,1,24.666667,14.000000,7.000000,3704.333333,28.055556,447.196667,13.333333
2326,618005e5f8090ec74528a44a,blue,6020bd98f1e4807cc700dc74,https://octane.gg/teams/dc74-team-bds,TEAM BDS,Europe,56.0,12.0,22.0,10.0,...,0,2,1,25.666667,15.000000,6.000000,3638.666667,22.962963,483.940000,12.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10544,62e02350da9d7ca1c7bb2f4b,blue,6020bd98f1e4807cc700dc74,https://octane.gg/teams/dc74-team-bds,TEAM BDS,Europe,38.0,11.0,25.0,8.0,...,0,2,1,37.333333,17.666667,8.000000,4487.333333,19.539171,796.820000,16.333333
10550,62e02350da9d7ca1c7bb2f4e,blue,6020bd98f1e4807cc700dc74,https://octane.gg/teams/dc74-team-bds,TEAM BDS,Europe,42.0,12.0,20.0,9.0,...,0,2,1,30.000000,16.666667,6.333333,3824.666667,20.299404,603.890000,12.666667
10582,62e02371da9d7ca1c7bb2f5e,blue,6020bd98f1e4807cc700dc74,https://octane.gg/teams/dc74-team-bds,TEAM BDS,Europe,54.0,15.0,28.0,14.0,...,0,2,1,33.666667,19.333333,9.000000,4540.666667,25.522139,639.263333,13.333333
10589,62e02371da9d7ca1c7bb2f61,orange,6020bd98f1e4807cc700dc74,https://octane.gg/teams/dc74-team-bds,TEAM BDS,Europe,61.0,18.0,29.0,14.0,...,1,2,1,44.666667,24.333333,12.666667,6123.333333,28.432192,886.033333,17.666667


In [28]:
matches_rolling = matches.groupby("team_name").apply(lambda x: rolling_averages(x, cols, new_cols))

/var/folders/zk/fp_x9fxd5bvc0s0t65kzck_m0000gq/T/ipykernel_13028/3985352770.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team_name").apply(lambda x: rolling_averages(x, cols, new_cols))


In [29]:
matches_rolling = matches_rolling.droplevel('team_name')

In [30]:
matches_rolling.index = range(matches_rolling.shape[0])

In [31]:
matches_rolling

,match_id,color,team_id,team_slug,team_name,team_region,core_shots,core_goals,core_saves,core_assists,...,color_code,region_code,target,core_shots_rolling,core_saves_rolling,core_goals_rolling,core_score_rolling,core_shooting_percentage_rolling,movement_time_supersonic_speed_rolling,demo_inflicted_rolling
0,628357d7da9d7ca1c7baf0ca,orange,628355d1da9d7ca1c7baf0b0,https://octane.gg/teams/f0b0-966,+966,Middle East & North Africa,32.0,7.0,30.0,4.0,...,1,3,0,37.333333,23.666667,8.666667,5079.000000,26.622370,844.733333,12.000000
1,628cdc02da9d7ca1c7bb0b84,orange,628355d1da9d7ca1c7baf0b0,https://octane.gg/teams/f0b0-966,+966,Middle East & North Africa,33.0,7.0,20.0,5.0,...,1,3,1,33.666667,25.666667,7.333333,5030.333333,25.386905,828.536667,15.000000
2,628cdc02da9d7ca1c7bb0b8b,orange,628355d1da9d7ca1c7baf0b0,https://octane.gg/teams/f0b0-966,+966,Middle East & North Africa,24.0,8.0,9.0,6.0,...,1,3,0,38.000000,26.333333,7.000000,4951.666667,19.124278,820.930000,14.666667
3,628cdc02da9d7ca1c7bb0b99,blue,628355d1da9d7ca1c7baf0b0,https://octane.gg/teams/f0b0-966,+966,Middle East & North Africa,43.0,6.0,15.0,4.0,...,0,3,0,29.666667,19.666667,7.333333,4080.000000,25.473485,727.933333,11.333333
4,61606299f8090ec74528700d,blue,613b735bf8090ec7452851a8,https://octane.gg/teams/51a8-00-nation,00 NATION,Europe,NaN,NaN,NaN,NaN,...,0,2,1,30.000000,23.333333,7.000000,4968.666667,23.174603,744.696667,19.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6446,616061f2f8090ec745286fdf,orange,61606469143c37878b2387b7,https://octane.gg/teams/87b7-zntra,ZNTRA,Asia-Pacific South,27.0,10.0,9.0,7.0,...,1,1,1,36.000000,22.666667,11.666667,5019.333333,32.085153,670.913333,8.000000
6447,61685f4af8090ec7452878a2,blue,61606469143c37878b2387b7,https://octane.gg/teams/87b7-zntra,ZNTRA,Asia-Pacific South,33.0,10.0,25.0,8.0,...,0,1,0,35.333333,15.000000,12.333333,4400.000000,35.235429,598.856667,7.000000
6448,61741c6c143c37878b23a766,blue,61606469143c37878b2387b7,https://octane.gg/teams/87b7-zntra,ZNTRA,Asia-Pacific South,36.0,12.0,14.0,9.0,...,0,1,1,32.000000,19.000000,11.000000,4754.000000,34.483726,753.663333,6.666667
6449,61741c6c143c37878b23a76b,orange,61606469143c37878b2387b7,https://octane.gg/teams/87b7-zntra,ZNTRA,Asia-Pacific South,NaN,NaN,NaN,NaN,...,1,1,0,32.000000,16.000000,10.666667,4489.666667,33.557800,737.786667,8.666667


In [32]:
def make_predictions(data, predictors):
    train = data[data["match_id"] < '62a3988cda9d7ca1c7bb22ba']
    test = data[data["match_id"] > '62a3988cda9d7ca1c7bb22ba']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], prediction = preds), index = test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision

In [33]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [34]:
precision

np.float64(0.7553191489361702)

In [35]:
combined

,actual,prediction
74,0,1
75,1,1
76,0,0
77,1,1
78,0,0
...,...,...
6062,1,1
6063,1,0
6064,0,0
6065,1,0


In [36]:
combined = combined.merge(matches_rolling[["match_id", "team_name", "winner"]], left_index = True, right_index = True)

In [37]:
combined

,actual,prediction,match_id,team_name,winner
74,0,1,62e022abc437fde7e02da87a,01 ESPORTS,False
75,1,1,62e022abc437fde7e02da883,01 ESPORTS,True
76,0,0,62e022abc437fde7e02da88a,01 ESPORTS,False
77,1,1,62e022abc437fde7e02da892,01 ESPORTS,True
78,0,0,62e022abc437fde7e02da895,01 ESPORTS,False
...,...,...,...,...,...
6062,1,1,62e022abc437fde7e02da890,VERSION1,True
6063,1,0,62e02350da9d7ca1c7bb2f4c,VERSION1,True
6064,0,0,62e02350da9d7ca1c7bb2f4e,VERSION1,False
6065,1,0,62e02350da9d7ca1c7bb2f51,VERSION1,True


In [38]:
merged = combined.merge(combined, left_on=["match_id"], right_on=["match_id"])

In [39]:
merged

,actual_x,prediction_x,match_id,team_name_x,winner_x,actual_y,prediction_y,team_name_y,winner_y
0,0,1,62e022abc437fde7e02da87a,01 ESPORTS,False,0,1,01 ESPORTS,False
1,0,1,62e022abc437fde7e02da87a,01 ESPORTS,False,1,1,KARMINE CORP,True
2,1,1,62e022abc437fde7e02da883,01 ESPORTS,True,1,1,01 ESPORTS,True
3,1,1,62e022abc437fde7e02da883,01 ESPORTS,True,0,0,GAIMIN GLADIATORS,False
4,0,0,62e022abc437fde7e02da88a,01 ESPORTS,False,0,0,01 ESPORTS,False
...,...,...,...,...,...,...,...,...,...
341,0,0,62e02350da9d7ca1c7bb2f4e,VERSION1,False,0,0,VERSION1,False
342,1,0,62e02350da9d7ca1c7bb2f51,VERSION1,True,0,0,SPACESTATION GAMING,False
343,1,0,62e02350da9d7ca1c7bb2f51,VERSION1,True,1,0,VERSION1,True
344,0,0,62e02371da9d7ca1c7bb2f60,VERSION1,False,1,1,FAZE CLAN,True


In [40]:
# Remove rows where team is versing themselves
merged.drop_duplicates(subset=['team_name_x', 'team_name_y'], inplace=True)

In [41]:
merged.index = range(merged.shape[0])

In [42]:
merged

,actual_x,prediction_x,match_id,team_name_x,winner_x,actual_y,prediction_y,team_name_y,winner_y
0,0,1,62e022abc437fde7e02da87a,01 ESPORTS,False,0,1,01 ESPORTS,False
1,0,1,62e022abc437fde7e02da87a,01 ESPORTS,False,1,1,KARMINE CORP,True
2,1,1,62e022abc437fde7e02da883,01 ESPORTS,True,0,0,GAIMIN GLADIATORS,False
3,0,0,62e022abc437fde7e02da88a,01 ESPORTS,False,1,1,DIGNITAS,True
4,1,1,62e022abc437fde7e02da892,01 ESPORTS,True,0,0,SENBEI STRIKERS,False
...,...,...,...,...,...,...,...,...,...
173,1,0,62e022abc437fde7e02da888,VERSION1,True,0,0,VELOCE ESPORTS,False
174,1,1,62e022abc437fde7e02da890,VERSION1,True,0,0,THE CLUB,False
175,1,0,62e02350da9d7ca1c7bb2f4c,VERSION1,True,0,0,NRG ESPORTS,False
176,0,0,62e02350da9d7ca1c7bb2f4e,VERSION1,False,1,1,TEAM BDS,True


In [43]:
merged[(merged["prediction_x"] == 1) & (merged["prediction_y"] == 0)]["actual_x"].value_counts()

actual_x
1    45
0     4
Name: count, dtype: int64

In [44]:
45/49

0.9183673469387755